Connecter un bot a internet pour récupérer la disponibilité d'une selection de film, le but était d'utilisé l'outil de google pour savoir si un films est dispo gratuitement ou abonnement sur une liste de service de streaming, envoyer une notif quand il est dispo sur ce service de streaming

<div class="ellip bclEt">Google Play Films et TV</div>
<div class="ellip rsj3fb">À partir de <span>2,99&nbsp;€</span></div>

<div class="ellip bclEt"> => avoir le nom du service de streaming,
<div class="ellip rsj3fb"> => avoir le prix du service et savoir si il est disponible en abonnement

In [51]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [6]:
#test request
def search_google(query):
    url = "https://www.google.com/search"
    params = {"q": query}
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }  # Ajoutez un en-tête User-Agent pour éviter le blocage par Google
    
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()  # Vérifiez si la requête a réussi
    
    return response.text   

In [81]:
def search_google(query):
    chrome_options = Options()
    s = Service('C:/Program Files (x86)/chromedriver')
    driver = webdriver.Chrome(service = s,options=chrome_options)
    driver.get("https://www.google.com/search?q=" + query)

    # Utilisez Selenium pour simuler l'appui sur la touche "Enter" afin d'effectuer la recherche
    #search_box = driver.find_element(By.NAME,"q")
    #search_box.send_keys(Keys.RETURN)
    
    driver.implicitly_wait(10)
    bouton_cookies = driver.find_element(By.ID, "L2AGLb")
    bouton_cookies.click()

    driver.implicitly_wait(10)
    button_xpath = '//g-expandable-content[@role="button" and @tabindex="0" and @jscontroller="Ah7cLd"]'
    button = driver.find_element(By.XPATH, button_xpath)
    button.click()
    
  
    
    # Récupérez le contenu HTML après que les contenus cachés soient disponibles
    time.sleep(2)
    html = driver.page_source
    
    driver.quit()  # Fermez le navigateur
    
    return html

In [82]:
def extract_div_elements(html, class_names):
    soup = BeautifulSoup(html, "html.parser")
    div_elements = soup.find_all("div", class_=class_names)
    
    for div in div_elements:
        print(div.text)

In [83]:
# Demande à l'utilisateur de saisir le terme de recherche
query = input("Entrez votre recherche : ")

# Envoie la requête de recherche et récupère les résultats
search_results = search_google(query)

# Extrait les éléments <div> ayant la classe "ellip bclEt"
extract_div_elements(search_results, ["ellip bclEt", "ellip rsj3fb"])

YouTube
À partir de 2,99 €
Google Play Films et TV
À partir de 2,99 €
CANAL VOD
À partir de 2,99 €
Amazon Prime Video
À partir de 2,99 €
Universciné
À partir de 2,99 €
Apple TV
À partir de 3,99 €
Netflix
Abonnement
